# New GDP Real-Time Dataset

**Author:** Jason Cruz  
**Last updated:** 08/13/2025  
**Python version:** 3.12  
**Project:** Rationality and Nowcasting on Peruvian GDP Revisions  

---
## 📌 Summary
This notebook documents the step-by-step **construction of real-time datasets** for **Peruvian GDP revisions** since 2013–PRESENT. It covers:

1. **Downloading PDFs** (actually Weekly Reports (WR)) from the Central Reserve Bank of Peru's website.
2. **Generating PDF inputs** by shorten them in order to keep key pages containing required tables where GDP growth rates are in.
3. **Cleaning-up data** extracted from input PDFs.
4. **Concatenating real-time datasets across years by frequency** 
5. **Storing RTD to SQL** for availability to users upon request and further analysis.

🌐 **Main Data Source:** [BCRP Weekly Report](https://www.bcrp.gob.pe/publicaciones/nota-semanal.html) (📰 WR, from here on)  
Any questions or issues regarding the coding, please email [Jason 📨](mailto:jj.cruza@up.edu.pe)  

---

## 🛠️ Libraries

If you don't have the libraries below, please use the following code (as example) to install the required libraries.

In [ ]:
#!pip install os # Comment this code with "#" if you have already installed this library.

Check out Python information

In [1]:
import sys
import platform

print("🐍 Python Information")
print(f"  Version  : {sys.version.split()[0]}")
print(f"  Compiler : {platform.python_compiler()}")
print(f"  Build    : {platform.python_build()}")
print(f"  OS       : {platform.system()} {platform.release()}")

🐍 Python Information
  Version  : 3.12.1
  Compiler : MSC v.1916 64 bit (AMD64)
  Build    : ('main', 'Jan 19 2024 15:44:08')
  OS       : Windows 10


**Import helper functions**

> ⚠️ Please, check the script `new_gdp_datasets_functions.py` which contains all the functions required by this _jupyter notebook_. The functions there are ordered according to the sections of this jupyter notebok.

In [2]:
from gdp_rtd_pipeline import *

pygame 2.5.2 (SDL 2.28.3, Python 3.12.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


## ⚙️ Initial set-up

Before preprocessing new GDP releases data, we will:

* **Create necessary folders** for storing inputs, outputs, logs, and screenshots.
* **Connect to the PostgreSQL database** containing GDP revisions datasets.
* **Import helper functions** from `new_gdp_datasets_functions.py`.

**Create necessary folders**

In [3]:
from pathlib import Path

PROJECT_ROOT = Path.cwd()
user_input = input("Enter relative path (default='.'): ").strip() or "."
target_path = (PROJECT_ROOT / user_input).resolve()
target_path.mkdir(parents=True, exist_ok=True)
print(f"📂 Using path: {target_path}")


Enter relative path (default='.'):  .


📂 Using path: C:\Users\Jason Cruz\OneDrive\Documentos\RA\CIUP\GDP Revisions\GitHub\peru_gdp_revisions\gdp_revisions_datasets


## 1. Downloading PDFs

Our main source for data collection is the [BCRP Weekly Report](https://www.bcrp.gob.pe/publicaciones/nota-semanal.html). The weekly report is a periodic (weekly) publication of the BCRP in compliance with article 84 of the Peruvian Constitution and articles 2 and 74 of the BCRP's organic law, which include, among its functions, the periodic publication of the main national macroeconomic statistics.
    
Our project requires the publication of **two tables**: the table of monthly growth rates of real GDP (12-month percentage changes), and the table of quarterly (annual) growth rates of real GDP. These tables are referred to as **Table 1** and **Table 2**, respectively, throughout this jupyter notebook.

### Scraper bot

This section automates the download of the **BCRP Weekly Report PDFs** directly from the official BCRP website.

**What it does:**
1. Opens the official BCRP Weekly Report page.
2. Finds and collects all PDF links.
3. Downloads them in chronological order (oldest to newest).
4. Optionally plays a notification sound every N downloads.
5. Organizes downloaded PDFs into year-based folders.

> 💡 If a CAPTCHA appears, solve it manually in the browser window and re-run the cell.

> 🔁 This script uses webdriver-manager to automatically handle browser drivers (default: Chrome), so you DO NOT need to manually download ChromeDriver, GeckoDriver, etc. If you want to change browser for your replication, modify the 'browser' parameter in init_driver().

> 🎵 Place your own MP3 file in `alert_track` folder for download notifications. Recommended free sources (CC0/public domain):
>  - Pixabay Audio: https://pixabay.com/music/
>  - FreeSound: https://freesound.org/
>  - FreePD: https://freepd.com/

In [7]:
# Define base folder for saving all digital PDFs
pdf_folder = 'pdf'

# Define subfolder for saving the original PDFs as downloaded from the BCRP website
raw_pdf_subfolder = os.path.join(pdf_folder, 'raw')

# Define subfolder for saving reduced PDFs containing only selected pages with GDP growth tables (monthly, quarterly, and annual frequencies)
input_pdf_subfolder = os.path.join(pdf_folder, 'input')

# Define folder for saving .txt files with download and dataframe record
record_folder = 'record'

# Define folder for saving warning bells. This is for download notifications (see section 1).
alert_track_folder = 'alert_track'

# Create all required folders (if they do not already exist) and confirm creation
for folder in [pdf_folder, raw_pdf_subfolder, input_pdf_subfolder, record_folder, alert_track_folder]:
    os.makedirs(folder, exist_ok=True)
    print(f"📂 {folder} created")

📂 pdf created
📂 pdf\raw created
📂 pdf\input created
📂 record created
📂 alert_track created


In [ ]:
# Run the function to start the scraper bot
pdf_downloader(
    bcrp_url = "https://www.bcrp.gob.pe/publicaciones/nota-semanal.html",
    raw_pdf_folder = raw_pdf_subfolder,
    download_record_folder = record_folder,
    download_record_txt = '1_downloaded_pdfs.txt',
    alert_track_folder = alert_track_folder,
    max_downloads = 60,
    downloads_per_batch = 6, 
    headless = False 
)

Probably the 📰 WR were downloaded in a single folder, but we would like the WR to be sorted by years. The following code sorts the PDFs into subfolders (years) for us by placing each WR according to the year of its publication. This happens in the **"blink of an eye"**.

Check your raw_pdf_subfolder out, every PDF should be placed in a year folder.

In [ ]:
# Get the list of files in the directory
files = os.listdir(raw_pdf_subfolder)

# Call the function to organize files
organize_files_by_year(raw_pdf_subfolder)

# WR-08-2017

This  is crucial for the upcoming steps, specially for the section 3, cleansing. If -in the future- you enconuter some issues by executing cleaing it is likely to atributte to the pdf nature. IN that case, you can return to this code to replace defectiv pdfs for those convinient ones

Don't worry about it...

Tú puedes hacer lo mismo si te enfrentas a un inconveniente similar. Incluso puedes descargar los casos excepecionales de WR de un mismo mes y reemplazar los defectuosos.

In [ ]:
# Replace specific defective PDFs (friendly outputs with icons)
replace_defective_pdfs(
    items=[
        ("2017", "ns-08-2017.pdf", "ns-07-2017"), # Enter the year (folder) that contains the defective PDF, the defective PDF, and the new chosen PDF 
        ("2019", "ns-23-2019.pdf", "ns-22-2019"), # The same one above
    ],
    root_folder=input_pdf_subfolder, # base folder with /2017, /2019, ...
    record_folder=record_folder, # folder with new_downloaded_pdfs.txt
    download_record_txt = '1_downloaded_pdfs.txt',
    quarantine=os.path.join(input_pdf_subfolder, "_quarantine")  # set to None to delete instead
)

## 2. Generating PDF inputs

Now that we have downloaded the 📰 WR from the Central Bank, we should know that each of these files has more than 100 pages, but not all of them contain the information required for this project.

All we really want is a couple of pages from each 📰 WR, one for **Table 1** (monthly real GDP growth) and one for **Table 2** (annual and quarterly real GDP growth). The code below is executed to maintain the **two key pages** with both tables of each PDF plus the cover page that contains the information that helps us identify one 📰 WR from another such as its date of publication and serial number.

_quarentine will be discard of the input PDF generator

In [ ]:
# Run the function to generate trimmed PDFs for input
pdf_input_generator(
    raw_pdf_folder = raw_pdf_subfolder,
    input_pdf_folder = input_pdf_subfolder,
    input_pdf_record_folder = record_folder,
    input_pdf_record_txt = '2_generated_input_pdfs.txt',
    keywords = ["ECONOMIC SECTORS"]
)

Again, probably the WR (PDF files, now of few pages) were stored in disorder in the `input_pdf_folder` folder. The following code sorts the PDFs into subfolders (years) by placing each WR (which now includes only the key tables) according to the year of its publication. This happens in the **"blink of an eye"**.  

In [ ]:
# Get the list of files in the directory
files = os.listdir(input_pdf_subfolder)

# Call the function to organize files
organize_files_by_year(input_pdf_subfolder)

## 3. Data cleaning

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px">
<p>     
Since we already have the PDFs <span style="font-size: 24px;">&#128462;</span> with just the tables required for this project, we can start extracting them. Then we can proceed with data cleaning.
</p>  
<div/>

### 3.2 Extracting tables and data cleanup

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px">
<p>     
The main library used for extracting tables from PDFs <span style="font-size: 24px;">&#128462;</span> is <code>pdfplumber</code>. You can review the official documentation by clicking <a href="https://github.com/jsvine/pdfplumber" style="color: rgb(0, 153, 123); font-size: 16px;">here</a>.
</p>
    
<p>     
    The functions in <b>Section 3</b> of the <code>"new_gdp_datasets_functions.py"</code> script were built to deal with each of these issues. An interesting exercise is to compare the original tables (the ones in the PDF <span style="font-size: 24px;">&#128462;</span>) and the cleaned tables (by the cleanup codes below). Thus, the cleanup codes for <a href="#3-2-1" style="color: rgb(0, 153, 123); font-size: 16px;">Table 1</a> and <a href="#3-2-1" style="color: rgb(0, 153, 123); font-size: 16px;">Table 2</a> generates two dictionaries, the first one stores the raw tables; that is, the original tables from the PDF <span style="font-size: 24px;">&#128462;</span> extracted by the <code>pdfplumber</code> library, while the second dictionary stores the fully cleaned tables.
</p>
<div/>

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px">
    The code iterates through each PDF <span style="font-size: 24px;">&#128462;</span> and extracts the two required tables from each. The extracted information is then transformed into dataframes and the columns and values are cleaned up to conform to Python conventions (pythonic).
    <div/>

<h3><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.2.1.</span>
    <span style = "color: dark; font-family: PT Serif Pro Book;">
    <span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">Table 1.</span> Extraction and cleaning of data from tables on monthly real GDP growth rates.
    </span>
    </h3>

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px">
<p>     
The basic criterion to start extracting tables is to use keywords (sufficient condition). I mean, tables containing the following keywords meet the requirements to be extracted.
</p>
<div/>

<div style="text-align: left;">
    <span style="font-size: 24px; color: rgb(255, 32, 78); font-weight: bold;">&#9888;</span>
    <span style="font-family: PT Serif Pro Book; color: black; font-size: 16px;">
        Please check that the flat file <b>"ns_dates.csv"</b> is updated with the dates, years and ids for the newly downloaded PDF <span style="font-size: 24px;">&#128462;</span> (WR). That file is located in the <b>"ns_dates"</b> folder and is uploaded to SQL from the jupyeter notebook <code>aux_files_to_sql.ipynb</code>
    </span>
</div>

Si por alguna razón ejecutas el código de la sección 3 y no continuas ejecutando la sección subsecuente, puedes estar tranquilo de que un registro los guardó. La próxima vez que visite este script basta con empezar desde esta sección 3 (eliminando el txt) para generar los dataframes que no se guardaron en ningun lado, estos son insumos esenciales para la sección 4. Alternativamente puede guardar todos los dataframes generados en una carpeta como respaldo y empezar desde la sección 4 cargándolos.

# Section 3 — Cleaning pipelines (Table 1 & Table 2)

### Functions
- `new_table_1_cleaner(...)` → cleans **Table 1 (monthly)** pages and returns two dicts:
  - `raw_tables_dict_1`: raw tables exactly as extracted from PDFs, keyed as `ns_xx_yyyy_1`.
  - `new_dataframes_dict_1`: cleaned tables ready for downstream steps, keyed as `ns_xx_yyyy_1`.

- `new_table_2_cleaner(...)` → cleans **Table 2 (quarterly/annual)** pages and returns:
  - `raw_tables_dict_2`: raw tables, keyed as `ns_xx_yyyy_2`.
  - `new_dataframes_dict_2`: cleaned tables, keyed as `ns_xx_yyyy_2`.

Both functions:
- **skip** year folder `_quarantine`
- maintain a **record txt** (chronologically sorted: year → issue)  
- show **Jupyter progress bars** (magenta = active, blue = finished)
- write a **log file**:
  - Table 1 → `logs/3_cleaner_1.log`
  - Table 2 → `logs/3_cleaner_2.log`

### Arguments
- `input_pdf_folder: str`  
  Root containing year subfolders with input PDFs (e.g., `input_pdf_subfolder/2017/ns-07-2017.pdf`).

- `record_folder: str`  
  Folder where the record txt is stored (e.g., `record/`).

- `record_txt: str` *(optional)*  
  Record filename. Defaults:
  - Table 1 → `new_generated_dataframes_1.txt`
  - Table 2 → `new_generated_dataframes_2.txt`

- `log_folder: str` *(optional, default `logs`)*  
  Where the `.log` files are written.

- `log_txt: str` *(optional)*  
  Log filename. Defaults:
  - Table 1 → `3_cleaner_1.log`
  - Table 2 → `3_cleaner_2.log`

- `persist: bool` *(optional, default `False`)*  
  If `True`, save cleaned tables to disk and update a manifest.  
  If `False`, nothing is saved (keeps repo light and re-runnable).

- `persist_folder: str | None` *(optional)*  
  Base folder for persisted outputs (default: `./data/clean`).  
  Layout when `persist=True`:
    data/clean/
    table_1/
    manifest.csv
    2017/
    ns-07-2017.parquet (or .csv if Parquet engine unavailable)
    table_2/
    manifest.csv
    2017/
    ns-07-2017.parquet

- `pipeline_version: str` *(optional, default `"s3.0.0"`)*  
Version tag recorded in `manifest.csv` for cache/audit. Bump it when the cleaning logic changes.

### Typical calls

```python
# Table 1 (monthly)
raw_1, clean_1 = new_table_1_cleaner(
  input_pdf_folder=input_pdf_subfolder,
  record_folder=record_folder,
  persist=True,                           # turn on checkpointing (Parquet/CSV + manifest)
  persist_folder=clean_data,              # e.g., os.path.join(project_root, "data", "clean")
  pipeline_version="s3.0.0"
)

# Table 2 (quarterly/annual)
raw_2, clean_2 = new_table_2_cleaner(
  input_pdf_folder=input_pdf_subfolder,
  record_folder=record_folder,
  persist=True,
  persist_folder=clean_data,
  pipeline_version="s3.0.0"
)



If you want the runners to *also* write the cleaned dicts out to a single combined Parquet/CSV per table (alongside the per-WR files), I can add that as an optional flag (`persist_combined=True`) without changing the defaults.


# If you will run until this section and you are planning to go back and retake from section 4, enter "True"

In [4]:
# Define base folder for saving vintages data (.csv)
data_folder = 'data'

# Define subfolder for saving 
input_data_subfolder = os.path.join(data_folder, 'input')

# Define subfolder for saving 
output_data_subfolder = os.path.join(data_folder, 'output')

# Create all required folders (if they do not already exist) and confirm creation
for folder in [data_folder, input_data_subfolder, output_data_subfolder]:
    os.makedirs(folder, exist_ok=True)
    print(f"📂 {folder} created")

📂 data created
📂 data\input created
📂 data\output created


In [8]:
raw_1, clean_1, vintages_1 = new_table_1_cleaner(
    input_pdf_folder = input_pdf_subfolder,
    record_folder = record_folder,
    record_txt = '3_created_vintages_tab_1.txt',
    persist = True,
    persist_folder = input_data_subfolder,
    pipeline_version = "s3.0.0",
)



🧹 Starting Table 1 cleaning...


📂 Processing Table 1 in 2024



🧹 2024:   0%|                                                                                                 …

✔️ 2024:   0%|                                                                                                …


⏩ 132 cleaned tables already generated for years: 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023

📊 Summary:

📂 12 folders (years) found containing input PDFs
🗃️ Already cleaned tables: 132
✨ Newly cleaned tables: 6
⏱️ 5 seconds


In [9]:
raw_1.keys()

dict_keys(['ns_04_2024_1', 'ns_08_2024_1', 'ns_11_2024_1', 'ns_15_2024_1', 'ns_19_2024_1', 'ns_23_2024_1'])

In [10]:
clean_1.keys()

dict_keys(['ns_04_2024_1', 'ns_08_2024_1', 'ns_11_2024_1', 'ns_15_2024_1', 'ns_19_2024_1', 'ns_23_2024_1'])

In [11]:
vintages_1.keys()

dict_keys(['ns_04_2024_1', 'ns_08_2024_1', 'ns_11_2024_1', 'ns_15_2024_1', 'ns_19_2024_1', 'ns_23_2024_1'])

In [12]:
raw_1['ns_11_2024_1']

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,2023,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,2024
0,SECTORES ECONÓMICOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ECONOMIC SECTORS
1,NaN,Ene.,Feb.,Mar.,Abr.,May.,Jun.,Jul.,Ago.,Sep.,Oct.,Nov.,Dic.,Año,Ene.
2,Agropecuario 2/,"3,6","-0,4","-1,0","-11,5","-4,9","-0,9","-0,9","-4,8","-7,6","-6,1","2,7","0,2","-2,9","-2,8 Agriculture and Livestock 2/"
3,Agrícola,"6,5","-0,6","0,2","-16,5","-6,4","-1,0","-1,2","-7,5","-11,9","-9,4","5,5","0,8","-4,1","-4,1Agriculture"
4,Pecuario,"-0,6","-0,2","-2,9","-0,4","-0,9","-0,6","-0,4","-0,7","-0,9","-0,8","-1,3","-0,6","-0,9","-0,9Livestock"
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Pesca,"27,4","1,0","16,5","-8,8","-71,3","-68,9","-47,9","49,1","16,9","51,6","61,0","-51,3","-19,7","-26,8 Fishing"
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Minería e hidrocarburos 3/,"-0,8","0,4","9,3","17,3","16,8","16,0","11,5","6,7","8,9","3,2","7,9","3,6","8,2","4,0 Mining and fuel 3/"
9,Minería metálica,"-0,3","2,6","8,7","20,8","21,0","19,1","13,4","6,3","7,4","3,3","10,4","4,1","9,5","4,6Metals"


In [13]:
clean_1['ns_11_2024_1']

,year,wr,sectores_economicos,economic_sectors,2023_ene,2023_feb,2023_mar,2023_abr,2023_may,2023_jun,2023_jul,2023_ago,2023_sep,2023_oct,2023_nov,2023_dic,2023_year,2024_ene
1,2024,11,agropecuario,agriculture and livestock,3.6,-0.4,-1.0,-11.5,-4.9,-0.9,-0.9,-4.8,-7.6,-6.1,2.7,0.2,-2.9,-2.8
2,2024,11,agricola,agriculture,6.5,-0.6,0.2,-16.5,-6.4,-1.0,-1.2,-7.5,-11.9,-9.4,5.5,0.8,-4.1,-4.1
3,2024,11,pecuario,livestock,-0.6,-0.2,-2.9,-0.4,-0.9,-0.6,-0.4,-0.7,-0.9,-0.8,-1.3,-0.6,-0.9,-0.9
4,2024,11,pesca,fishing,27.4,1.0,16.5,-8.8,-71.3,-68.9,-47.9,49.1,16.9,51.6,61.0,-51.3,-19.7,-26.8
5,2024,11,mineria e hidrocarburos,mining and fuel,-0.8,0.4,9.3,17.3,16.8,16.0,11.5,6.7,8.9,3.2,7.9,3.6,8.2,4.0
6,2024,11,mineria metalica,metals,-0.3,2.6,8.7,20.8,21.0,19.1,13.4,6.3,7.4,3.3,10.4,4.1,9.5,4.6
7,2024,11,hidrocarburos,fuel,-3.5,-11.5,12.7,-0.3,-4.5,-0.7,-1.6,9.1,19.6,2.8,-8.0,-0.1,0.7,0.6
8,2024,11,manufactura,manufacturing,0.2,-1.6,-0.4,-3.7,-15.7,-14.6,-13.4,-4.1,-9.3,-2.7,-0.5,-10.9,-6.6,-4.2
9,2024,11,procesadores recursos primarios,based on raw materials,12.3,22.9,27.0,12.4,-28.2,-29.1,-18.4,15.9,8.3,9.3,18.6,-27.8,-1.8,-17.9
10,2024,11,manufactura no primaria,nonprimary,-4.1,-8.8,-7.3,-8.3,-10.3,-8.0,-11.5,-9.5,-13.8,-6.0,-7.0,-2.9,-8.2,1.5


In [ ]:
vintages_1['ns_11_2024_1']

# Checking the cleaning version out

In [ ]:
df100 = vintages_1["ns_04_2022_1"]
print(df100.attrs)
# {'pipeline_version': 's3.0.0'}


In [ ]:
vintages_1["ns_04_2022_1"].attrs

<h3><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.2.2.</span>
    <span style = "color: dark; font-family: PT Serif Pro Book;">
    <span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">Table 2.</span> Extraction and cleaning of data from tables on quarterly and annual real GDP growth rates.
    </span>
    </h3>

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px">
<p>     
The basic criterion to start extracting tables is to use keywords (sufficient condition). I mean, tables containing the following keywords meet the requirements to be extracted.
</p>
<div/>

<div style="text-align: left;">
    <span style="font-size: 24px; color: rgb(255, 32, 78); font-weight: bold;">&#9888;</span>
    <span style="font-family: PT Serif Pro Book; color: black; font-size: 16px;">
        Please check that the flat file <b>"ns_dates.csv"</b> is updated with the dates, years and ids for the newly downloaded PDF <span style="font-size: 24px;">&#128462;</span> (WR). That file is located in the <code>ns_dates</code> folder and is uploaded to SQL from the jupyeter notebook <code>aux_files_to_sql.ipynb</code>
    </span>
</div>

In [ ]:
raw_2, clean_2, vintages_2 = new_table_2_cleaner(
    input_pdf_folder = input_pdf_subfolder,
    record_folder = record_folder,
    record_txt = '3_created_vintages_tab_2.txt',
    persist = True,
    persist_folder = input_data_subfolder,
    pipeline_version = "s3.0.0",
)


In [ ]:
raw_2['ns_04_2022_2']

In [ ]:
clean_2['ns_04_2022_2']

In [ ]:
vintages_2['ns_04_2022_2']

In [ ]:
df200 = vintages_2["ns_04_2022_1"]
print(df200.attrs)
# {'pipeline_version': 's3.0.0'}


In [ ]:
vintages_2["ns_04_2022_1"].attrs

## 4. GDP Real-Time dataset

**Connect to the PostgreSQL database**

The following function will establish a connection to the `gdp_revisions_datasets` database in `PostgreSQL`. The **input data** used in this jupyter notebook will be loaded from this `PostgreSQL` database, and similarly, all **output data** generated by this jupyter notebook will be stored in that database. Ensure that you set the necessary parameters to access the server once you have obtained the required permissions.

> 💡 **Tip:** To request permissions, please email [Jason 📨](mailto:jj.cruza@alum.up.edu.pe)  
> ⚠️ **Warning:** Make sure you have set your SQL credentials as environment variables before proceeding.  

In [ ]:
from sqlalchemy import create_engine
import os

In [ ]:
def create_sqlalchemy_engine(database="gdp_revisions_datasets", port=5432):
    """
    Create an SQLAlchemy engine to connect to the PostgreSQL database.
    
    Environment Variables Required:
        CIUP_SQL_USER: SQL username
        CIUP_SQL_PASS: SQL password
        CIUP_SQL_HOST: SQL host address

    Args:
        database (str): Name of the database. Default is 'gdp_revisions_datasets'.
        port (int): Port number. Default is 5432.

    Returns:
        engine (sqlalchemy.engine.Engine): SQLAlchemy engine object.
    
    Raises:
        ValueError: If required environment variables are missing.

    Example:
        engine = create_sqlalchemy_engine()
    """
    user = os.environ.get('CIUP_SQL_USER')
    password = os.environ.get('CIUP_SQL_PASS')
    host = os.environ.get('CIUP_SQL_HOST')

    if not all([host, user, password]):
        raise ValueError("❌ Missing environment variables: CIUP_SQL_HOST, CIUP_SQL_USER, CIUP_SQL_PASS")

    connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"
    engine = create_engine(connection_string)

    print(f"🔗 Connected to PostgreSQL database: {database} at {host}:{port}")
    return engine

In [ ]:
engine = create_sqlalchemy_engine()

In [6]:
concatenated_1 = concatenate_table_1(
    input_data_subfolder = input_data_subfolder,
    record_folder = record_folder,
    record_txt = '4_concatenated_vintages_tab_1.txt',
    persist = True,
    persist_folder = input_data_subfolder
)


🧹 Starting Table 1 concatenation...
📂 Batch 1 (2013-2022) saved to data\input\new_gdp_rtd_table_1_batch_1.csv
📂 Batch 2 (2023-2024) saved to data\input\new_gdp_rtd_table_1_batch_2.csv

📊 Summary:
📂 12 folders (years) found containing input CSVs
🗃️ Already processed files: 0
✨ Newly concatenated files: 138
⏱️ 1 seconds
🔹 Total batches created: 2 with ranges: ['new_gdp_rtd_table_1_batch_1', 'new_gdp_rtd_table_1_batch_2']


In [7]:
concatenated_1.keys()

dict_keys(['new_gdp_rtd_table_1_batch_1', 'new_gdp_rtd_table_1_batch_2'])

In [8]:
concatenated_1['new_gdp_rtd_table_1_batch_1']

,target_period,agriculture_2013_1,agriculture_2013_2,agriculture_2013_3,agriculture_2013_4,agriculture_2013_5,agriculture_2013_6,agriculture_2013_7,agriculture_2013_8,agriculture_2013_9,...,services_2022_3,services_2022_4,services_2022_5,services_2022_6,services_2022_7,services_2022_8,services_2022_9,services_2022_10,services_2022_11,services_2022_12
0,2011m11,4.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011m12,-1.3,-1.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012m1,2.9,2.4,2.4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2012m2,2.9,3.1,3.1,3.1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2012m3,2.2,2.2,2.2,2.2,2.2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,2023m12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146,2024m1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147,2024m2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
148,2024m3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
concatenated_2 = concatenate_table_2(
    input_data_subfolder = input_data_subfolder,
    record_folder = record_folder,
    record_txt = '4_concatenated_vintages_tab_2.txt',
    persist = True,
    persist_folder = input_data_subfolder
)


🧹 Starting Table 2 concatenation...
📂 Batch 1 (2013-2022) saved to data\input\new_gdp_rtd_table_2_batch_1.csv
📂 Batch 2 (2023-2024) saved to data\input\new_gdp_rtd_table_2_batch_2.csv

📊 Summary:
📂 12 folders (years) found containing input CSVs
🗃️ Already processed files: 0
✨ Newly concatenated files: 138
⏱️ 2 seconds
🔹 Total batches created: 2 with ranges: ['new_gdp_rtd_table_2_batch_1', 'new_gdp_rtd_table_2_batch_2']


In [ ]:
concatenated_2.head()